In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio autoplay="" src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" loop controls>

In [ ]:
#@title Install Backend & Deps
!apt update
!apt install aria2 -y
!apt install -y build-essential cmake
!pip install py-localtunnel

# Clone the repository
!git clone https://github.com/ggml-org/llama.cpp.git
%cd llama.cpp

# Optional: set environment variable to enable CUDA support (for GPU)
import os
os.environ["LLAMA_CUBLAS"] = "1"  # Optional: remove if using CPU

# Build the project (detects LLAMA_CUBLAS if set)
!make server


In [ ]:
#@title Download Model
OPTIONS = {
    1: {
        "id": "mradermacher/Fimbulvetr-11B-v2-GGUF",
        "file": "https://huggingface.co/mradermacher/Fimbulvetr-11B-v2-GGUF/blob/main/Fimbulvetr-11B-v2.Q5_K_S.gguf"
    },
    2: {
        "id": "mradermacher/MN-LooseCannon-12B-v2-GGUF",
        "file": "https://huggingface.co/mradermacher/MN-LooseCannon-12B-v2-GGUF/blob/main/MN-LooseCannon-12B-v2.Q6_K.gguf"
    },
    3: {
        "id": "bartowski/Dans-PersonalityEngine-V1.1.0-12b-GGUF",
        "file": "https://huggingface.co/bartowski/Dans-PersonalityEngine-V1.1.0-12b-GGUF/blob/main/Dans-PersonalityEngine-V1.1.0-12b-Q6_K.gguf"
    },
    4: {
        "id":"mradermacher/Magnum-Picaro-0.7-v2-12b-i1-GGUF",
        "file":"https://huggingface.co/mradermacher/Magnum-Picaro-0.7-v2-12b-i1-GGUF/blob/main/Magnum-Picaro-0.7-v2-12b.i1-Q6_K.gguf"
    },
    5: {
        "id":"mradermacher/patricide-12B-Unslop-Mell-GGUF",
        "file": "https://huggingface.co/mradermacher/patricide-12B-Unslop-Mell-GGUF/blob/main/patricide-12B-Unslop-Mell.Q6_K.gguf"
    }
}

PRESET = "5" #@param [1,2,3,4,5]

MODEL = OPTIONS[int(PRESET)]["file"].replace("/blob/", "/resolve/")

!aria2c -x 10 -o models/model.gguf --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none $MODEL

In [ ]:
#@LAUNCH!

MODEL_PATH = r"models/model.gguf"

GPU_LAYERS = 80 #@param
CONTEXT_SIZE = 8192 #@param

!./server -m $MODEL_PATH --host 0.0.0.0 --port 8000 --threads 4 --ctx-size $CONTEXT_SIZE --gpu-layers $GPU_LAYERS --mlock --log-disable & pylt port 8000
